# Halite Simulator

In this Kernel you will find halite environment simulator. It is:
* ~10 times more performant
* 100% matches behavior of kaggle_environments
* Compatible with all agents and Halite renderer

Creating this simulator was an interesting excercise. There is already similar [Simulator](https://www.kaggle.com/elvenmonk/high-performance-rps-dojo) I made for [Rock Paper Scissors](https://www.kaggle.com/c/rock-paper-scissors) environment.

I can see how "Simulator" can be used/modified for efficient forward search.

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import time

actions = [
    'CONVERT',
    'NONE',
    'EAST',
    'WEST',
    'NORTH',
    'SOUTH',
]
action_map = {
    'CONVERT': 0,
    'EAST': 2,
    'WEST': 3,
    'NORTH': 4,
    'SOUTH': 5,
}
N_ACTIONS = 6

def plot(state: np.ndarray, title = None, subtitles = None, size=5, vmax=1):
    D = state.ndim
    R = 1 if D < 4 else state.shape[-4]
    C = 1 if D < 3 else state.shape[-3]
    plt.figure(figsize=(C*size, R*size))
    if title is not None:
        plt.suptitle(title)
    for r in range(R):
        for c in range(C):
            plt.subplot(R, C, r * C + c + 1)
            X = state.shape[-1]
            Y = state.shape[-2]
            board = state if D == 2 else state[c] if D == 3 else state[r][c]
            subtitle = None if subtitles is None else subtitles if D == 2 else subtitles[c] if D == 3 else subtitles[r][c]
            plt.imshow(board, cmap='binary', vmax=vmax, extent=(0, X, Y, 0))
            ax = plt.gca();
            ax.set_xticks(range(0,X+1), minor=True)
            ax.set_yticks(range(0,Y+1), minor=True)
            ax.set_xticks(range(0,X+1,3))
            ax.set_yticks(range(0,Y+1,3))
            ax.set_xticklabels(range(0,X+1,3))
            ax.set_yticklabels(range(0,Y+1,3))
            ax.grid(which='both', color='k', linestyle='-', linewidth=1)
            if subtitle is not None:
                plt.title(subtitle)
    plt.show()

# Define simulator

In this section we will build Simulator step-by-step and validate each step, using sample replay from the top of [Halite by Two Sigma - Playground Edition](https://www.kaggle.com/c/halite-iv-playground-edition) leaderboard, which I exposed as [Public Dataset](https://www.kaggle.com/elvenmonk/sample-halite-4-playground-replay)

## Init configuration
We can assume configuration doesn't change over time, so it is used to initialize configuration

In [ ]:
class Simulator:
    def __init__(self, configuration, N_PLAYERS):
        self.configuration = configuration
        self.N_PLAYERS = N_PLAYERS
        self.player_data = [[0, {}, {}] for _ in range(self.N_PLAYERS)]

        # Halite configuration
        self.START_HALITE = configuration['startingHalite']
        self.MAX_HALITE = configuration['maxCellHalite']
        self.COST_SPAWN = configuration['spawnCost']
        self.COST_CONVERT = configuration['convertCost']
        self.RATE_COLLECT = configuration['collectRate']
        self.RATE_REGEN = configuration['regenRate']

        # Board dimentions
        self.N_CELLS = configuration['size']
        self.SIZE_DATA = (self.N_CELLS, self.N_CELLS)
        self.SIZE_PLAYER_DATA = (self.N_PLAYERS, self.N_CELLS, self.N_CELLS)
        self.SIZE_ACTION_DATA = (self.N_PLAYERS, N_ACTIONS, self.N_CELLS, self.N_CELLS)
        
        # We need our simulator to produce same replay as original kaggle_environment would do.
        # We can not use np.round because it can give different result in some corner cases when value is close to or is exactly half integer.
        # Read more on differences: https://numpy.org/doc/stable/reference/generated/numpy.around.html#numpy.around
        # We also have to convert numpy float32/float64 type to python float before rounding, otherwise result can be different in some cases
        # Coming up with this approach was the most trickiest part of the simulator
        self.round3 = np.frompyfunc(lambda x: round(float(x) * (1 + self.RATE_REGEN), 3), 1, 1)
        
        # Board data
        self.player_halite = np.zeros(self.N_PLAYERS)
        self.board_halite = np.zeros(self.SIZE_DATA)
        self.yards = np.zeros(self.SIZE_PLAYER_DATA, dtype=bool)
        self.ships = np.zeros(self.SIZE_PLAYER_DATA, dtype=bool)
        self.ship_halite = np.zeros(self.SIZE_PLAYER_DATA)

        # Action data
        self.yard_actions = np.zeros(self.SIZE_PLAYER_DATA, dtype=bool)
        self.ship_actions = np.zeros(self.SIZE_ACTION_DATA, dtype=bool)

        # Intermediate data
        self.next_places = np.zeros(self.SIZE_ACTION_DATA, dtype=bool)
        self.next_halite = np.zeros(self.SIZE_ACTION_DATA, dtype=int)

## Load replay

Let's load replay and use it to initialize our simulator

In [ ]:
with open('../input/sample-halite-4-playground-replay/8603037.json') as f:
    replay_data = json.load(f)

N_PLAYERS = len(replay_data['info']['TeamNames'])
PLOT_MAX_HALITE = replay_data['configuration']['maxCellHalite'] + replay_data['configuration']['convertCost']
simulator = Simulator(replay_data['configuration'], N_PLAYERS)

replay_steps = replay_data['steps']
replay_length = len(replay_steps)

player_legends = [f'Player {p}' for p in range(N_PLAYERS)]
action_legends = [[f'Player {p} {actions[a]}' for a in range(N_ACTIONS)] for p in range(N_PLAYERS)]

## Parse observation

We can reset our simulator to any valid observation

In [ ]:
def parse(self, observation):
    self.board_halite.flat[:] = observation['halite']
    self.yards = np.zeros(self.SIZE_PLAYER_DATA, dtype=bool)
    self.ships = np.zeros(self.SIZE_PLAYER_DATA, dtype=bool)
    self.ship_halite = np.zeros(self.SIZE_PLAYER_DATA)
    player_data = observation['players']
    for p in range(self.N_PLAYERS):
        self.player_halite[p] = player_data[p][0]
        self.player_data[p][0] = player_data[p][0]
        self.player_data[p][1] = player_data[p][1]
        self.player_data[p][2] = player_data[p][2]
        self.yards[p].flat[list(self.player_data[p][1].values())] = True
        for v in self.player_data[p][2].values():
            self.ships[p].flat[v[0]] = True
            self.ship_halite[p].flat[v[0]] = v[1]

setattr(Simulator, 'parse', parse)

simulator.parse(replay_steps[0][0]['observation'])

plot(simulator.board_halite, 'Board halite', vmax=simulator.MAX_HALITE)
plot(simulator.yards, 'Shipyards', player_legends)
plot(simulator.ships, 'Ships', player_legends)
plot(simulator.ship_halite, 'Ship halite', player_legends, vmax=PLOT_MAX_HALITE)

## Parse actions

When working with replay, we need to load actions from step sata.

In [ ]:
def parse_actions(self, step_data):
    self.yard_actions = np.zeros(self.SIZE_PLAYER_DATA, dtype=bool)
    self.ship_actions = np.zeros(self.SIZE_ACTION_DATA, dtype=bool)
    for p in range(self.N_PLAYERS):
        action_data = step_data[p]['action']
        for key in self.player_data[p][1]:
            if key in action_data and action_data[key] == 'SPAWN':
                self.yard_actions[p].flat[self.player_data[p][1][key]] = True
        for key in self.player_data[p][2]:
            a = 1
            if key in action_data and action_data[key] in action_map:
                a = action_map[action_data[key]]
            self.ship_actions[p][a].flat[self.player_data[p][2][key][0]] = True

setattr(Simulator, 'parse_actions', parse_actions)

simulator.parse_actions(replay_steps[1])

plot(simulator.yard_actions, 'Shipyard SPAWN actions', player_legends)
plot(simulator.ship_actions, 'Ship actions', action_legends)

# Turn Resolution

> After both players have submitted their actions for the turn, the system will automatically resolve the turn and update the board state. Both player turns are resolved simultaneously across the following phases:



## 1. Spawning

> All shipyards that ordered a spawn action are resolved with new ships being added to the board on top of the shipyards that spawned them. At this phase multiple ships/shipyards can occupy the same space on the board- these overlaps will be resolved in collision resolution. Spawning is resolved in board position order (top left first), which becomes relevant if a player runs out of halite part way through resolving all of their spawning orders.

Successful spawn can be obtained by clipping cumulative sum of spawns, and using diff to restore original spawn positions.

In [ ]:
def step1_spawn(self):
    temp1 = self.yards & self.yard_actions
    temp2 = temp1 * self.COST_SPAWN
    temp3 = np.cumsum(temp2.reshape((self.N_PLAYERS, -1)), axis=1).reshape(self.SIZE_PLAYER_DATA)
    temp4 = temp3 <= self.player_halite[:,None,None]
    temp5 = temp1 & temp4

    self.next_places[:,0] = temp5
    self.player_halite -= np.sum(temp5, axis=(1, 2)) * self.COST_SPAWN

setattr(Simulator, 'step1_spawn', step1_spawn)

simulator.step1_spawn()

print(simulator.player_halite)
plot(simulator.next_places[:,0], 'Ships SPAWNed', player_legends)

## 2. Conversion

> All ships that attempted to convert into shipyards are resolved. Ships turn into shipyards if there are sufficient funds available and they are not already on top of an existing shipyard. Shipyard conversions are resolved in board position order (top left first), which becomes relevant if a player runs out of halite part way through resolving all of their conversion orders.

From [source code](https://github.com/Kaggle/kaggle-environments/blob/b7882beacb45aa9b46d66da4e83460b36a641199/kaggle_environments/envs/halite/helpers.py#L755):
> Excess halite leftover from conversion is added to the player's total only after all conversions have completed
> This is to prevent the edge case of chaining halite from one convert to fund other converts

In [ ]:
def step2_convert(self):
    temp1 = ~self.yards & self.ships & self.ship_actions[:, 0]
    temp2 = temp1 * np.maximum(0, self.COST_CONVERT - self.ship_halite)
    temp3 = np.cumsum(temp2.reshape((self.N_PLAYERS, -1)), axis=1).reshape(self.SIZE_PLAYER_DATA)
    temp4 = temp3 <= self.player_halite[:,None,None]
    temp5 = temp1 & temp4

    self.yards[:] = self.yards | temp5
    self.player_halite -= np.sum(temp5, axis=(1, 2)) * self.COST_CONVERT
    self.player_halite += np.sum(temp5 * self.ship_halite, axis=(1, 2))
    self.ship_halite *= ~temp5
    self.board_halite *= ~np.any(temp5, axis = 0)

setattr(Simulator, 'step2_convert', step2_convert)

simulator.step2_convert()

print(simulator.player_halite)
plot(simulator.yards, 'Shipsyards', player_legends)

## 3. Movement

> All ships are moved according to their orders. Note that ships are able to move “through” each other (if they each move to the other’s previous space) without colliding.

In [ ]:
def step3_move(self):
    temp1 = self.ships[:,None] & self.ship_actions
    temp2 = temp1 * self.ship_halite[:,None]

    self.next_places[:,1] = temp1[:, 1]
    self.next_places[:,2] = np.roll(temp1[:, 2], +1, axis=2)
    self.next_places[:,3] = np.roll(temp1[:, 3], -1, axis=2)
    self.next_places[:,4] = np.roll(temp1[:, 4], -1, axis=1)
    self.next_places[:,5] = np.roll(temp1[:, 5], +1, axis=1)
    self.next_halite[:,1] = temp2[:, 1]
    self.next_halite[:,2] = np.roll(temp2[:, 2], +1, axis=2)
    self.next_halite[:,3] = np.roll(temp2[:, 3], -1, axis=2)
    self.next_halite[:,4] = np.roll(temp2[:, 4], -1, axis=1)
    self.next_halite[:,5] = np.roll(temp2[:, 5], +1, axis=1)

setattr(Simulator, 'step3_move', step3_move)

simulator.step3_move()

plot(simulator.next_places, 'New ship positions', action_legends)
plot(simulator.next_halite, 'Ship halite', action_legends, vmax=PLOT_MAX_HALITE)

## 4. Ship Collision

> Ship collision is resolved, reducing the number of ships in every cell on the board to one or less. In each collision the smallest ship is the survivor, with all others destroyed. The smallest ship is defined as the ship with the least halite in its storage. If multiple ships tie for having the least halite, all the ships in the collision are destroyed. Note that collision does not consider which player owns which ships, collision between friendly ships is possible. The surviving ships takes all of the halite from the destroyed ships’ cargoes and retains its place on the board.

In [ ]:
def step4_collide(self):
    temp1 = np.min(self.next_halite + (~self.next_places) * self.START_HALITE, axis = (0, 1))
    temp2 = self.next_places & (self.next_halite == temp1)
    temp3 = np.sum(temp2, axis = (0, 1)) == 1

    self.ships[:] = np.sum(temp2 & temp3, axis = 1) == 1
    self.ship_halite[:] = self.ships * np.sum(self.next_halite, axis = (0, 1))

setattr(Simulator, 'step4_collide', step4_collide)

simulator.step4_collide()

plot(simulator.ships, 'Ships', player_legends)
plot(simulator.ship_halite, 'Ship halite', player_legends, vmax=PLOT_MAX_HALITE)

## 5. Shipyard Collision

> If a ship and shipyard from different players occupy the same cell on the board both are destroyed. Any halite held by the ships is lost. Having your shipyard destroyed does not reduce your total halite, all deposited halite has already been teleported safely back to your homeworld.

In [ ]:
def step5_crash(self):
    temp1 = np.any(self.yards & ~self.ships & np.any(self.ships, axis=0), axis=0)

    self.yards &= ~temp1
    self.ships &= ~temp1
    self.ship_halite *= ~temp1

setattr(Simulator, 'step5_crash', step5_crash)

simulator.step5_crash()

plot(simulator.yards, 'Yards', player_legends)
plot(simulator.ships, 'Ships', player_legends)
plot(simulator.ship_halite, 'Ship halite', player_legends, vmax=PLOT_MAX_HALITE)

## 6. Halite Depositing

> All ships that are on top of friendly shipyards deposit all halite they have in their individual storage.

In [ ]:
def step6_deposit(self):
    temp1 = self.yards & self.ships

    self.player_halite += np.sum(temp1 * self.ship_halite, axis=(1, 2))
    self.ship_halite *= ~temp1

setattr(Simulator, 'step6_deposit', step6_deposit)

simulator.step6_deposit()

print(simulator.player_halite)
plot(simulator.ship_halite, 'Ship halite', player_legends, vmax=PLOT_MAX_HALITE)

## 7. Halite Mining

> All ships that held their position on top of halite mine it, moving 25% of the cell’s halite into that ship’s storage. Halite in ship storage is indicated in the visualizer by a blue glow on top of the ship.

In [ ]:
def step7_mine(self):
    temp1 = self.ships & self.ship_actions[:, 1]
    temp2 = (temp1 * self.board_halite * self.RATE_COLLECT).astype(int)

    self.ship_halite += temp2
    self.board_halite -= np.sum(temp2, axis=0)

setattr(Simulator, 'step7_mine', step7_mine)

simulator.step7_mine()

plot(simulator.board_halite, 'Board halite', vmax=simulator.MAX_HALITE)
plot(simulator.ship_halite, 'Ship halite', player_legends, vmax=PLOT_MAX_HALITE)

## 8. Halite Regeneration

> Every cell on the board that has more than 0 halite and no ships on top of it will now regenerate by 2% of the existing amount of halite in the cell. Halite can grow up to a maximum of 500 halite per cell.

In [ ]:
def step8_regen(self):
    temp1 = np.any(self.ships, axis=0)

    self.board_halite[:] = np.where(temp1, self.board_halite, np.minimum(self.round3(self.board_halite), self.MAX_HALITE))

setattr(Simulator, 'step8_regen', step8_regen)

simulator.step8_regen()

plot(simulator.board_halite, 'Board halite', vmax=simulator.MAX_HALITE)

## Observation players data

In order to get next actions from agents, we need to convert data back to observation format.
Note: current version does not preserve ship and yard IDs between steps. This is enough for simple agents but can break agents that rely on consistent IDs.
Rendering also relies on IDs to be preserved, so you can see "explosion" artifacts on replays produced from Simulator.
I hope to fix it in next version.

In [ ]:
def encode_data(self):
    for p in range(self.N_PLAYERS):
        indices = np.flatnonzero(self.yards[p])
        self.player_data[p][0] = int(self.player_halite[p])
        self.player_data[p][1] = { f'{i+1}-{p+1}': int(indices[i]) for i in range(len(indices)) }
        indices = np.flatnonzero(self.ships[p])
        self.player_data[p][2] = { f'{i+1}-{p+1}': [int(indices[i]), int(self.ship_halite[p].ravel()[indices[i]])] for i in range(len(indices)) }

setattr(Simulator, 'encode_data', encode_data)

simulator.encode_data()

print(simulator.player_data)

## All together

Now we can combine entire turn resolution into single method

In [ ]:
def next_step(self, step_data):
    self.parse_actions(step_data)
    self.step1_spawn()
    self.step2_convert()
    self.step3_move()
    self.step4_collide()
    self.step5_crash()
    self.step6_deposit()
    self.step7_mine()
    self.step8_regen()
    self.encode_data()
    
setattr(Simulator, 'next', next_step)

## End Turn

> The “step” number is incremented, ending the turn. The updated board is saved and redistributed to each agent as the “observation”, moving to the next turn.

> At this step a player is eliminated if they are no longer able to viably compete in the game. This happens when the player no longer has any ships and has less than 500 halite remaining or no remaining shipyards (and thus cannot spawn any more ships).

> At the end of each turn the game also checks if either player has been eliminated or if the game has reached turn 400. If either of those is true instead of starting a new turn the game moves to the game end step.

In [ ]:
def end_turn(self, step, step_data):
    for p in range(self.N_PLAYERS):
        if step_data[p]['status'] != 'ACTIVE':
            continue
        step_data[p]['observation'] = {
            'halite': list(self.board_halite.ravel()),
            'players': self.player_data,
            'player': p,
            'remainingOverageTime': 60,
            'step': step,
        }
        if not self.player_data[p][2] and (self.player_data[p][0] < self.COST_SPAWN or not self.player_data[p][1]):
            step_data[p]['status'] = 'DONE'
            step_data[p]['reward'] = step - self.N_STEPS - 1
        else:
            step_data[p]['reward'] = self.player_data[p][0]
    return [utils.structify(player_data) for player_data in step_data]

setattr(Simulator, 'end_turn', end_turn)

# Validation

First let's define method that compares next replay step data with data generated by Simulator. This is to ensure that Simulator is 100% accurate

In [ ]:
def validate(self, observation, verbose = False):
    valid = True
    player_data = observation['players']
    for p in range(N_PLAYERS):
        valid &= np.all(self.player_halite[p] == np.array(player_data[p][0]))
        if verbose:
            print(f'Player {p} halite:', ['Error','OK'][valid * 1])
        if not valid:
            print(self.player_data[p][0], np.array(player_data[p][0]))
        valid &= np.all(np.sort(np.array(list(self.player_data[p][1].values()))) == np.sort(np.array(list(player_data[p][1].values()))))
        if not valid:
            print(self.player_data[p][1], player_data[p][1])
        self.player_data[p][1] = player_data[p][1]
        mask = np.zeros(self.yards[p].shape, bool)
        mask.flat[list(self.player_data[p][1].values())] = True
        valid &= np.all(self.yards[p] == mask)
        if verbose:
            print(f'Player {p} yards:', ['Error','OK'][valid * 1])
        if not valid:
            print(self.yards[p], mask)
        mask = np.zeros(self.ships[p].shape, bool)
        mask_halite = np.zeros(self.ship_halite[p].shape)
        valid &= np.all(np.sort(np.array(list(self.player_data[p][2].values())), axis=0) == np.sort(np.array(list(player_data[p][2].values())), axis=0))
        if not valid:
            print(self.player_data[p][2], player_data[p][2])
        self.player_data[p][2] = player_data[p][2]
        for v in self.player_data[p][2].values():
            mask.flat[v[0]] = True
            mask_halite.flat[v[0]] = v[1]
        valid &= np.all(self.ships[p] == mask)
        if verbose:
            print(f'Player {p} ships:', ['Error','OK'][valid * 1])
        if not valid:
            print(self.ships[p], mask)
        valid &= np.all(self.ship_halite[p] == mask_halite)
        if verbose:
            print(f'Player {p} ship halite:', ['Error','OK'][valid * 1])
        if not valid:
            print(self.ship_halite[p], mask_halite)
    halite_data = np.array(observation['halite']).reshape(self.board_halite.shape)
    valid &= np.all(np.abs(self.board_halite - halite_data) < 1e-15)
    if verbose:
        print('Board halite:', ['Error','OK'][valid * 1])
    if not valid:
        print(self.board_halite - halite_data)
    return valid

setattr(Simulator, 'validate', validate)

simulator.validate(replay_steps[1][0]['observation'], True)

## Validate replay

Now let's simulate every step of replay and compare to replay data. If they match then we know that Simulator implementation matches kaggle_environment.

In [ ]:
simulator.parse(replay_steps[0][0]['observation'])
for t in tqdm(range(1,replay_length), total=replay_length-1):
    step_data = replay_steps[t]
    simulator.next(step_data)
    if not simulator.validate(step_data[0]['observation']):
        print(f'Failed on step {t}')
        break
print('Done!')

# Benchmark

We still need to improve our Simulator to use it interchangably with kaggle_environments

## Run agents

Now let's use Simulator to play new match instead of simulating existing replay. We can implement method similar to "env.run"

In [ ]:
def run(self, agents):
    self.N_STEPS = self.configuration['episodeSteps']
    steps = [[]] * self.N_STEPS
    step_data = [{
        'action': {},
        'info': {},
        'observation': {},
        'reward': 0,
        'status': 'ACTIVE',
    } for p in range(self.N_PLAYERS)]

    for step in range(self.N_STEPS):
        steps[step] = self.end_turn(step, step_data)
        remaining = sum(1 for p in range(self.N_PLAYERS) if step_data[p]['status'] == 'ACTIVE')
        if remaining < 2:
            break
        for p in range(self.N_PLAYERS):
            step_data[p]['action'] = agents[p](step_data[p]['observation'], self.configuration)
        self.next(step_data)
    for p in range(self.N_PLAYERS):
        steps[step][p]['status'] = 'DONE'
    return steps[:step+1]

setattr(Simulator, 'run', run)

## Sample submission

This simple random agent is stable enough to be used in our tests.

In next version Simulator will generate same spip and yard IDs and we will be able to cache responses returned on first Simulator pass and return same cached responses to following Simulator and Kaggle environment runs (to make runs identical for true benchmark).

In [ ]:
%%writefile submission.py

from random import choice

def get_pos(N, pos, a):
    return (pos % N + (-1 if a == "WEST" else 1 if a == "EAST" else 0)) + N * ((pos // N + (1 if a == "SOUTH" else -1 if a == "NORTH" else 0)) % N)

def agent(observation, configuration):
    actions = {}
    try:
        N_CELLS = configuration['size']
        COST_SPAWN = configuration['spawnCost']
        COST_CONVERT = configuration['convertCost']
        p = observation['player']
        halite, yard_data, ship_data = observation['players'][p]
        halite_data = observation['halite']
        ship_hash = {ship_data[k][0]:k for k in ship_data}

        for yard_id in yard_data:
            action = choice(["SPAWN", None])
            if halite >= COST_SPAWN and action is not None:
                actions[yard_id] = action
                halite -= COST_SPAWN
                ship_hash[yard_data[yard_id]] = yard_id

        for ship_id in ship_data:
            ship_pos, ship_halite = ship_data[ship_id]
            if halite_data[ship_pos] < COST_CONVERT/2:
                if not yard_data or ship_halite >= COST_SPAWN + COST_CONVERT:
                    actions[ship_id] = "CONVERT"
                else:
                    a = choice(["NORTH", "SOUTH", "EAST", "WEST"])
                    if get_pos(N_CELLS, ship_pos, a) not in ship_hash:
                        actions[ship_id] = a
    except Exception as ex:
        print(ex)
    return actions

## Load Kaggle environment

We will use same board and agents for both Kaggle environment and Simulator runs.

In [ ]:
%run submission.py

from kaggle_environments import evaluate, make, utils
agents = ['/kaggle/working/submission.py'] * N_PLAYERS
legends = [{ 'name': a.split('/')[-1] } for a in agents]
env = make('halite', configuration={ 'episodeSteps': 400 }, info={ 'EpisodeId': 0, 'TeamNames': legends })

## Execute Simulator

In [ ]:
%%time
agents = [agent] * N_PLAYERS
state = env.reset(N_PLAYERS)
simulator = Simulator(env.configuration, N_PLAYERS)
simulator.parse(state[0]['observation'])
env.steps = simulator.run(agents)

## Render result

In [ ]:
env.render(mode="ipython", width=800, height=400, agents=legends, autoplay=True, controls=True)

## Execute Kaggle Enviromnent

In [ ]:
%%time
state = env.reset(len(agents))
result = env.run(agents)

## Render

In [ ]:
env.render(mode="ipython", width=800, height=400, agents=legends)

# Conclusion

As shown above Halite rules can be fully vectorized using Numpy. Potentially evaluation performance can be further improved using GPU, parallel computations, etc.
Additional optimizations possible if multiple similar initial states are evaluated to get similar next steps (e.g. if forward moves space search is performed).

I hope you find this Kernel useful.

I hope also that we will see Halive V+ competitions and new rules will be still compatible with Numpy.